# Harmoniser les noms de locuteurs

## Les enjeux de l’harmonisation

Le manque d’uniformité dans l’écriture du nom des locuteurs soulève de nombreux problèmes, notamment dans l’attribution des répliques.

In [ ]:
t1 = "Viviane\tDe quoi?"
t2 = "Dame du Lac\tÉcoutez, je suis désolée de vous déranger en plein milieu de vos obligations mais c'est urgent."
t3 = "La Dame du Lac\tEn fait ce serait pour une petite mission... mais je voudrais pas vous surcharger de travail, vous m'avez l'air bien à bloc déjà..."

*Viviane*, *Dame du Lac* et *La Dame du Lac*, qui pourtant désignent une même entité, seront considérés par une machine comme des personnes différentes.

In [ ]:
speakers = set()    # A collection of unique items
for text in [t1, t2, t3]:
    splitted = text.split('\t')    # Tab helps to split the line in two fragments
    speaker = splitted[0]          # The speaker is the first fragment
    speakers.add(speaker)          # Adds the speaker to the collection
nb = len(speakers)                 # 3

Et si l’on veut comptabiliser le nombre de répliques par locuteur :

In [ ]:
stats = {}

# For each speaker in the list
for speaker in speakers:
    stats.update(
        {
            # If the key already exists, add + 1 to nb of cues
            # if not, initializes it at 0
            speaker: stats.get(speaker, 0) + 1
        }
    )
print(stats)

## Identifier les sources d’erreurs

De nombreux facteurs président à la multiplication des formes d’un même locuteur :
- capitalisation (LA DAME DU LAC, La dame du Lac, La Dame du lac)
- fautes de frappes (Gueniève, Arhur)
- inconstances (Lancelot, Lancelot du Lac)
- mauvaises graphies (Démétra, Leodagan)
- méconnaissance des usages (Le Roi Burgonde, Le Chef Viking)
- fautes de transcriptions (Ferbach au lieu de Fearmac)
- ignorance du nom de certains personnages (Torri, Ferghus)
- …

## Établir une notice d’autorité

Une notice d’autorité des noms de personnages a pour objectif d’identifier sans ambiguïté les différents locuteurs.

La [fiche *Wikipédia*](https://fr.wikipedia.org/wiki/Personnages_de_Kaamelott) constitue une bonne entrée en matière.

À l’aide des notions abordées dans la partie sur le *Web scraping*, on obtient rapidement une liste des personnages principaux de la série.

In [ ]:
speakers = []
with open('./files/characters.txt') as file:
    for line in file:
        speakers.append(line.strip())
print(speakers[:10])

Il reste à compléter la liste avec toutes les formes rejetées, en séparant chaque valeur par une tabulation `\t`.

```txt
Aélis    Aelis    Aels  
Angharad    Angarade    Angharade
```

La première valeur de chaque ligne correspond à la forme retenue, toutes les autres sont les formes rejetées.

Le recueil des formes rejetées se fait par un algorithme qui va détecter la présence, pour chaque ligne du script analysé, d’une forme de locuteur qui n’apparaît pas déjà dans la liste des personnages.

Un arbitrage manuel est nécessaire pour inscrire cette forme soit comme forme retenue, soit comme forme rejetée ou soit comme la forme d’un personnage qui n’est pas encore listé.

La lecture d’un fichier au format tabulé est simplifiée par le module `csv` de Python :

In [ ]:
import csv
speakers = []
with open('./files/authorities.txt') as file:
    characters = csv.reader(file, delimiter="\t")
    [speakers.append(character) for character in characters]   
print(speakers[61])

Le type de données `set` permet d’obtenir une liste unique de valeurs :

In [ ]:
speakers_in_script = set()
with open('./sample/S03E82-witness.txt') as file:
    script = csv.reader(file, delimiter="\t")
    [speakers_in_script.add(line[0]) for line in script]
print(speakers_in_script)

L’objectif maintenant est de déterminer si l’autorité *Le maître d’armes* est maintenue au détriment des formes rencontrées dans le script (*Le maître d’arme*, *Maitre d'armes*).

Si c’est le cas, ces dernières seront rajoutées comme formes rejetées de *Le maître d’armes* dans la notice d’autorité. L’entrée correspondante deviendra :

```txt
Le maître d’armes    Le maître d’arme    Maitre d'armes
```

## Comparer deux ensembles

L’un des avantages des `set` en Python est de pouvoir effectuer dessus des opérations ensemblistes (inclusion, intersection…) :

In [ ]:
authorities = set(['Arthur Pendragon', 'Le maître d’armes', 'Lancelot du Lac', 'Perceval de Galles'])
speakers = set(['Arthur Pendragon', 'Perceval de Galles', 'Lancelot'])

print(speakers - authorities)     # Difference

De là, on peut écrire un algorithme pour comparer toutes les formes listées dans la notice d’autorité des personnages avec les formes effectivement présentes des locuteurs dans les scripts.

## Corriger les formes rejetées

La procédure consiste à comparer la forme d’un locuteur avec les formes de la notice d’autorité des personnages :
- s’il s’agit de la forme retenue, elle est conservée ;
- s’il s’agit d’une forme rejetée, elle est remplacée par la forme retenue.

Considérons la liste d’autorité suivante :

In [ ]:
authorities = [
    ['Arthur Pendragon', 'Arhur', 'ARTHUR', 'Arthur'],
    ['Lancelot du Lac', 'Lancelot'],
    ['Le maître d’armes', 'Le maître d’arme', 'Maître d’armes'],
    ['Léodagan']
]

Et la liste des formes de locuteurs d’un script :

In [ ]:
speakers_in_script = ['Arthur', 'Lancelot', 'Maître d’arme', 'Le maître d’arme', 'Léodagan']

Une façon de procéder consisterait à, pour chaque locuteur (Arthur, Lancelot…), parcourir la liste d’autorité et, si la forme (Arthur) est différente de celle retenue (Arthur Pendragon) mais qu’elle figure parmi les formes rejetées (Arhur, ARTHUR, Arthur), alors la remplacer par la forme retenue.

In [ ]:
# For each speaker in the script…
for speaker in speakers_in_script:
    print("Current form:", speaker)
    # … browses the authorities
    for authority in authorities:
        # If the speaker is in the authorities
        # but it is also different than the correct form
        if speaker in authority and speaker != authority[0]:
            # Replaces it with the correct form
            speaker = authority[0]
            print("Correct form:", speaker)

Ce genre de procédé mobilise beaucoup de ressources : la liste d’autorité est chargée dans son ensemble pour chaque locuteur !

Le type de données `dict` (dictionnaire) en Python permet d’améliorer sensiblement la procédure.

Un dictionnaire permet d’associer une valeur à une clé : un numéro de téléphone à un nom, un code postal à une ville…

In [ ]:
codes = {
    'Paris': '75000',
    'Nantes': '44000'
}
print(codes['Paris'])

Grâce à ce principe, on peut créer un dictionnaire qui, pour chaque forme rejetée d’un locuteur, renvoie la forme retenue :

In [ ]:
replacements = {}
for authority in authorities:
    valid_form = authority[0]
    invalid_forms = authority[1:]
    for form in invalid_forms:
        replacements.update(
            {
                form: valid_form
            }
        )
print(replacements)

Ensuite, pour chaque locuteur présent dans un script de la série, on interroge uniquement la clé correspondante dans le dictionnaire :

In [ ]:
for speaker in speakers_in_script:
    print("current form:", speaker)
    speaker = replacements[speaker]
    print("corrected form:", speaker)

Comme la forme *Maître d’arme* ne figure pas encore dans la notice d’autorité, une exception est levée et empêche la suite du traitement.

Ce problème ne devrait pas se produire avec une notice d’autorité complète et à jour. Malgré tout, il existe une méthode liée aux dictionnaires pour ne pas interrompre l’exécution du script en présence d’une clé manquante, `get()` :

In [ ]:
for speaker in speakers_in_script:
    print("current form:", speaker)
    # If speaker is a valid key, proceed; if not, let speaker as the corrected form
    speaker = replacements.get(speaker, speaker)
    print("corrected form:", speaker)